In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# 작업 경로를 MyDrive로 변경하여 구글 드라이브에 접속 후 바로 작업 디렉토리를 확인 가능

print('현재 작업 경로 :', os.getcwd())
os.chdir('/content/drive/MyDrive')
print('변경된 작업 경로 :', os.getcwd())

현재 작업 경로 : /content/drive/MyDrive/dataset
변경된 작업 경로 : /content/drive/MyDrive


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="RoWuJWQ3RKo2rQDMuRxf")
project = rf.workspace("ai-project-wuvsy").project("car-models-r4egq")
dataset = project.version(5).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Car-Models-5 in yolov5pytorch:: 100%|██████████| 4878/4878 [00:29<00:00, 167.62it/s]


In [ ]:
# 필요한 패키지 다운로드 및 import
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import yaml
from IPython.display import Image, clear_output

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

/content/drive/MyDrive/yolov5
Setup complete. Using torch 2.0.1+cu118 (Tesla T4)


In [ ]:
# 사용할 데이터셋 경로 및 데이터셋의 yaml 파일 경로 지정
data_dir = '/content/drive/MyDrive/dataset/Car-Models-5'
data_yaml = '/content/drive/MyDrive/dataset/Car-Models-5/data.yaml'

In [ ]:
# 데이터셋 yaml 파일 확인
with open(data_yaml) as f:
    film = yaml.load(f, Loader=yaml.FullLoader)
    display(film)

{'names': ['car', 'cargo'],
 'nc': 2,
 'roboflow': {'license': 'CC BY 4.0',
  'project': 'car-models-r4egq',
  'url': 'https://universe.roboflow.com/ai-project-wuvsy/car-models-r4egq/dataset/5',
  'version': 5,
  'workspace': 'ai-project-wuvsy'},
 'test': '../test/images',
 'train': '/content/drive/MyDrive/dataset/Car-Models-5/train/images',
 'val': '/content/drive/MyDrive/dataset/Car-Models-5/valid/images'}

In [ ]:
# yaml 파일의 train, val 데이터가 있는 경로 수정 (기존 경로 -> 구글 드라이브에 저장된 경로로)
film['train'] = '/content/drive/MyDrive/dataset/Car-Models-5/train/images'
film['val'] = '/content/drive/MyDrive/dataset/Car-Models-5/valid/images'

with open(data_yaml, 'w') as f:
    yaml.dump(film, f)

print('변경된 yaml 파일 :')
with open(data_yaml) as f:
    film = yaml.load(f, Loader=yaml.FullLoader)
    display(film)

변경된 yaml 파일 :


{'names': ['car', 'cargo'],
 'nc': 2,
 'roboflow': {'license': 'CC BY 4.0',
  'project': 'car-models-r4egq',
  'url': 'https://universe.roboflow.com/ai-project-wuvsy/car-models-r4egq/dataset/5',
  'version': 5,
  'workspace': 'ai-project-wuvsy'},
 'test': '../test/images',
 'train': '/content/drive/MyDrive/dataset/Car-Models-5/train/images',
 'val': '/content/drive/MyDrive/dataset/Car-Models-5/valid/images'}

In [ ]:
!python train.py --img 416 --batch 16 --epochs 150 --data '/content/drive/MyDrive/dataset/Car-Models-5/data.yaml' --weights yolov5s.pt --cache

WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")

train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/dataset/Car-Models-5/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=150, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, lab

In [ ]:
# 테스트 이미지 경로
test_data_dir = film['val']

# 실험 번호 지정 - 진행한 실험이 저장된 경로 확인해서 기입! - runs/train/exp1 인지 exp2, exp3 인지 확인 후 아래 번호 지정
#train_exp_num = 5
train_exp_num = 8

In [ ]:
#!python detect.py --weights runs/train/exp{train_exp_num}/weights/best.pt --img 416 --conf 0.1 --source {test_data_dir}
# roboflow에서 만든 모델의 test/images 경로
#!python detect.py --weights runs/train/exp/weights/best.pt --img 416 --conf 0.1 --source '/content/drive/MyDrive/dataset/Car Models.v2i.yolov5pytorch/test/images'
!python detect.py --weights runs/train/exp{train_exp_num}/weights/best.pt --img 416 --conf 0.1 --source '/content/drive/MyDrive/dataset/Car-Models-5/test/images'

WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")

detect: weights=['runs/train/exp8/weights/best.pt'], source=/content/drive/MyDrive/dataset/Car-Models-5/test/images, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=Fal

In [ ]:
# 테스트 결과 확인해보기


import glob
from IPython.display import Image, display

#runs/detect/exp 번호
#test_exp_num = 6
test_exp_num = 9


if not os.path.exists('/content/drive/MyDrive/yolov5/runs/detect/exp' + str(test_exp_num) + '/') :
  raise Exception('test_exp_num 을 다시 확인하세요.')

for imageName in glob.glob('/content/drive/MyDrive/yolov5/runs/detect/exp' + str(test_exp_num) + '/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

Output hidden; open in https://colab.research.google.com to view.